In [1]:
import asn1tools
import glob
import copy

In [2]:
OUTPUT_FORMATS = ("ber", 'xer', 'per', 'jer', 'uper', 'gser')
msg_spec = asn1tools.parse_files("../asn1-trustmessages/messages.asn")
fmt_spec = asn1tools.parse_files("../asn1-trustmessages/formats.asn")
codecs = {
    "msg": {f: asn1tools.compile_dict(msg_spec, f) for f in OUTPUT_FORMATS},
    "fmt": {f: asn1tools.compile_dict(fmt_spec, f) for f in OUTPUT_FORMATS}
}
messages = asn1tools.compile_files("../asn1-trustmessages/messages.asn", 'ber')
formats = asn1tools.compile_files("../asn1-trustmessages/formats.asn", 'ber')

In [3]:
def determine_target(m):
    if m['payload'][1]['format'] == "1.1.1":
        return 'QTM'
    elif m['payload'][1]['format'] == "2.2.2":
        return 'SL'
    else:
        raise Error("Unknown type")

In [4]:
def decode(data, codecs, enc):
    m = codecs["msg"][enc].decode('Message', data)
    
    if 'payload' in m and 'data-response' in m['payload']:
        target = determine_target(m)

        for item in m["payload"][1]["response"]:
            item["value"] = formats.decode(target, item["value"])
    return m

In [5]:
def encode(message, codecs, enc):
    if 'payload' in message and 'data-response' in message['payload']:
        m = copy.deepcopy(message)
        target = determine_target(m)
        for item in m["payload"][1]["response"]:
            item["value"] = codecs["fmt"][enc].encode(target, item["value"])
    else:
        m = message
    
    return codecs["msg"][enc].encode('Message', m)

In [6]:
files = glob.iglob("../*.ber")
for filename in files:
    enc = filename.split(".")[-1]
    name = filename.rsplit(".", 1)[:-1][0]
    
    # print(filename, enc, name)
    
    with open(filename, "rb") as handle:
        data = handle.read()   
    
    message = decode(data, codecs, enc)
    
    for codec_name in OUTPUT_FORMATS:
        if enc == codec_name:
            print("Skipping", codec_name, "for", filename)
            continue
        
        new_file = name + "." + codec_name
        with open(new_file, "wb") as handle:
            print("Writing", new_file)
            encoded = encode(message, codecs, codec_name)
            handle.write(encoded)

Skipping ber for ../message-data-request-30.ber
Writing ../message-data-request-30.xer
Writing ../message-data-request-30.per
Writing ../message-data-request-30.jer
Writing ../message-data-request-30.uper
Writing ../message-data-request-30.gser
Skipping ber for ../message-data-response-100.ber
Writing ../message-data-response-100.xer
Writing ../message-data-response-100.per
Writing ../message-data-response-100.jer
Writing ../message-data-response-100.uper
Writing ../message-data-response-100.gser
Skipping ber for ../message-data-request-100.ber
Writing ../message-data-request-100.xer
Writing ../message-data-request-100.per
Writing ../message-data-request-100.jer
Writing ../message-data-request-100.uper
Writing ../message-data-request-100.gser
Skipping ber for ../message-data-request-70.ber
Writing ../message-data-request-70.xer
Writing ../message-data-request-70.per
Writing ../message-data-request-70.jer
Writing ../message-data-request-70.uper
Writing ../message-data-request-70.gser
Sk